# U-Net Predict

Script to use the U-Net trained model to predict upon unseen data, in order to generate the deforestation detection map.

## 1. Importing libraries and changing working directory

Reference for the libraries:

+ [numpy](https://numpy.org/)
+ [gdal](https://gdal.org/api/python.html)
+ [os](https://docs.python.org/3/library/os.html)
+ [deepgeo](https://github.com/rvmaretto/deepgeo)

In [ ]:
import numpy as np
import gdal
import os
import deepgeo.common.geofunctions as gf
import deepgeo.common.visualization as vis
import deepgeo.dataset.dataset_generator as dg
import deepgeo.dataset.preprocessor as prep
import deepgeo.dataset.sequential_chips as seqchips
import deepgeo.networks.model_builder as mb

In [ ]:
# folder where all data is stored
os.chdir(os.getcwd().rsplit('/',2)[0]+'/Data')

## 2. Defining parameters and data pre-processing

In [ ]:
# id of the trained model
model_id = '2021-07-07_16-59-24'
model_path = f'./models/UNET/{model_id}'

In [ ]:
# approach for the training samples
sit = 'appr1'
# cell to predict upon
cell = '089098'
# cell used as training samples
cell_training = '089098'
# year
year = '2019'

In [ ]:
#########################
raster_file = f'./predictions/LSTM/{sit}.{year}.{cell}.tr{cell}.PS.tif'
preproc = prep.Preprocessor(raster_file, no_data=-9999)
preproc.set_nodata_value(-1)
preproc.standardize_image('norm_range', {"min":-1, "max":1})
raster_array = preproc.get_array_stacked_raster()

params_cg = {'raster_array': raster_array,
             'overlap': [186, 186],
             'win_size': 1084}

chip_struct = seqchips.SequentialChipGenerator(params_cg).generate_chips()

In [ ]:
vis.plot_image_histogram(raster_array, title="Normalized Raster", legend=['LSTM Prediction',
                                                                          'SRTM'])

In [ ]:
vis.plot_chips(chip_struct, raster_array, bands=[0,0,1], contrast=True)

## 3. Making the prediction

In [ ]:
model = mb.ModelBuilder(model_path)
chips = model.predict(chip_struct, model_dir=model_path)

## 4. Saving result

In [ ]:
if not os.path.exists('./predictions/UNET'):
    os.makedirs('./predictions/UNET')

output_pred = f"./predictions/UNET/{sit}.{year}.{cell}.tr{cell_training}.tif"
raster_array = None
preproc = None
gf.write_pred_chips(output_pred, raster_file, chips, chip_key='predict')

In [ ]:
print(output_pred)

In [ ]:
pred_rarr = gdal.Open(output_pred).ReadAsArray()
pred_rarr = np.asarray(pred_rarr, dtype=np.float32)
pred_rarr[pred_rarr==0] = None

import matplotlib.pyplot as plt

plt.figure(figsize=(10,10))
plt.imshow(pred_rarr, cmap='Set1', interpolation='nearest')
plt.colorbar()
plt.show()